In [ ]:
import pandas as pd
import random as rndm
#dataset_link="Bike Sharing Original Data.csv" # для чтения с диска
dataset_link=r"https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Bike%20Sharing%20Original%20Data.csv" # для чтения напрямую с репозитория
df0=pd.read_csv(dataset_link, sep=",")
# Проанализировав датасет, можно заметить (например выполнив запрос df['Year'].value_counts()), что он содержит данные за 2011 и 2012 годы.
# Разделяем датасет на части
df2011=df0[df0['Year']==2011]
df2012=df0[df0['Year']==2012]

# Собираем обратно
df=pd.concat([df2011, df2012])

#Проверка на дубликаты и их удаление в случае наличия
dups=df.duplicated().sum()
print (f"Найдено {dups} дубликатов")
if dups>0:
    df=df.drop_duplicates()
    print ("Произведено удаление дубликатов")
print("")

#Проверка на пропуски
print (f"Проверка на пропуски (начальная):\n{df.isna().sum()}\n")

#Добавление пропусков
n=32
r_max=df.shape[0] - 1
c_max=df.shape[1] - 1
while n>0:
    r=rndm.randint(0, r_max)
    c=rndm.randint(0, c_max)
    df.iloc[r, c]=None
    n-=1
print("Пропуски добавлены\n")

#Повторная проверка на пропуски
print (f"Проверка на пропуски:\n{df.isna().sum()}\n")

#Заполнение пропусков для столбца Wind Speed 
ws=df['Wind Speed'].mean()
print (f"Средняя скорость ветра: {ws:.2f}")
df['Wind Speed']=df['Wind Speed'].fillna(ws)
print("Пропуски в столбце Wind Speed заполнены\n")

#Заполнение пропусков для столбца Weather Situation 
ws_mode=df['Weather Situation'].mode().values[0]
print (f"Наиболее частые погодные условия: {ws_mode}")
df['Weather Situation']=df['Weather Situation'].fillna(ws_mode)

print("Пропуски в столбце Weather Situation заполнены\n")

#Повторная проверка на пропуски
print (f"Проверка на пропуски:\n{df.isna().sum()}\n")

#Удаление оставшихся пропусков
df.dropna(inplace=True)
print("Оставшиеся пропуски удалены\n")

#Повторная проверка на пропуски
print (f"Проверка на пропуски:\n{df.isna().sum()}\n")

#Поиск выбросов
print("Проверка столбца Wind Speed на выбросы")
q1=df['Wind Speed'].quantile(0.25)
q3=df['Wind Speed'].quantile(0.75)
d=1.5*(q3-q1)
lower_bound=q1-d
upper_bound=q3+d
print(f"нижняя граница: {lower_bound:.2f}")
print(f"верхняя граница: {upper_bound:.2f}")
outlires=df[(df['Wind Speed']<lower_bound)|(df['Wind Speed']>upper_bound)] # строго
outlires_count=outlires.shape[0]
print(f"Найдено выбросов: {outlires_count}\n")

if outlires_count>0:
    #Удаление выбросов
    df=df[(df['Wind Speed']>=lower_bound)&(df['Wind Speed']<=upper_bound)] # включая концы
    print("Выбросы удалены\n")

#Нормализация в столбце Temperature
print("Нормализация данных в столбце Temperature")
t_min=df['Temperature'].min()
t_max=df['Temperature'].max()
print(f"минимум: {t_min:.2f}")
print(f"максимум: {t_max:.2f}")
df['Temperature (norm)']=(df['Temperature'] - t_min)/(t_max - t_min)
print("\nОписание итогового датафрейма:")
df.describe()